# Preparando o Ambiente


In [45]:
!pip install pyspark #instala o pyspark no ambiente do google colab

from pyspark.sql import SparkSession # Importar a SparkSession

from pyspark.sql.functions import col, year, to_date, count, when
from pyspark.sql.types import IntegerType
# - col: para referenciar uma coluna
# - IntegerType: para definir o tipo "número inteiro"
# - to_date, year: para manipular datas
# - count, when: para verificar nulos

#cria a SparkSession
spark = SparkSession.builder \
    .appName("Tratamento_Youtube") \
    .getOrCreate()

print("SparkSession iniciada!")
print(spark)



SparkSession iniciada!


# Ler 'vídeos-stats.csv'

In [46]:
df_video = spark.read.csv("/content/videos-stats.csv", header=True, inferSchema=True) # Lê o 1º arquivo (videos-stats.csv) para o DataFrame df_video, usando cabeçalho e inferindo os tipos de dados

print("Leitura do 'video-stats.csv' concluída. Schema:")
df_video.printSchema() # Exibe a estrutura inicial (colunas e tipos) do df_video

Leitura do 'video-stats.csv' concluída. Schema:
root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Comments: double (nullable = true)
 |-- Views: double (nullable = true)



# Alterar valores nulos para 0

In [47]:
df_video = df_video.na.fill(0, subset=['Likes','Comments','Views']) # Preenche valores nulos (NULL) com o número 0 nas colunas estatísticas de interação (Likes, Comments, Views)

print("valores nulos (NULL) nos campos Likes, Comments e Views preenchidos com 0.")

valores nulos (NULL) nos campos Likes, Comments e Views preenchidos com 0.


# Ler 'comments.csv'

In [48]:
df_comentario = spark.read.csv("/content/comments.csv", header=True, inferSchema=True) # Lê o 2º arquivo (comments.csv) para o DataFrame df_comentario

print("Leitura do 'comments.csv' concluída. Schema:")
df_comentario.printSchema()

Leitura do 'comments.csv' concluída. Schema:
root
 |-- _c0: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Likes: string (nullable = true)
 |-- Sentiment: string (nullable = true)



# Calcular quantidade de registros

In [ ]:
# .count() é uma ação que força o Spark a ler e contar as linhas
count_video_inicial = df_video.count() # Calcula o número total de linhas no df_video antes da limpeza
count_comentario_inicial = df_comentario.count() # Calcula o número total de linhas no df_comentario antes da limpeza

print(f"Quantidade de registros em 'df_video' (antes): {count_video_inicial}")
print(f"Quantidade de registros em 'df_comentario' (antes): {count_comentario_inicial}")

Quantidade de registros em 'df_video' (antes): 1869
Quantidade de registros em 'df_comentario' (antes): 22555


#  Remover registros com 'Video ID' nulos

In [ ]:
# .na.drop() remove linhas inteiras se o valor for nulo
# Usei 'subset' para focar apenas na coluna 'Video ID'
df_video = df_video.na.drop(subset=["Video ID"])
df_comentario = df_comentario.na.drop(subset=["Video ID"])

# Recalcular a contagem para ver quantos registros saíram
count_video_sem_nulos = df_video.count()
count_comentario_sem_nulos = df_comentario.count()

print(f"Registros removidos de 'df_video' (Video ID nulo): {count_video_inicial - count_video_sem_nulos}")
print(f"Quantidade de registros em 'df_video' (agora): {count_video_sem_nulos}")
print(f"Registros removidos de 'df_comentario' (Video ID nulo): {count_comentario_inicial - count_comentario_sem_nulos}")
print(f"Quantidade de registros em 'df_comentario' (agora): {count_comentario_sem_nulos}")

Registros removidos de 'df_video' (Video ID nulo): 12
Quantidade de registros em 'df_video' (agora): 1869
Registros removidos de 'df_comentario' (Video ID nulo): 7481
Quantidade de registros em 'df_comentario' (agora): 22555


# Remover 'Video ID' duplicados

In [49]:
# .dropDuplicates() remove linhas duplicadas
# Usei 'subset' para definir a "chave" de duplicidade
df_video = df_video.dropDuplicates(subset=["Video ID"])

count_video_sem_duplicatas = df_video.count()
print(f"Registros duplicados removidos de 'df_video': {count_video_sem_nulos - count_video_sem_duplicatas}")
print(f"Quantidade de registros em 'df_video'(final): {count_video_sem_duplicatas}")


Registros duplicados removidos de 'df_video': 0
Quantidade de registros em 'df_video'(final): 1869


# Converter tipos em 'df_video'

In [50]:
# Usei .withColumn() para "substituir" uma coluna
# Usei .cast() para forçar o tipo para IntegerType (Inteiro)
df_video = df_video.withColumn("Likes", col("Likes").cast(IntegerType()))
df_video = df_video.withColumn("Comments", col("Comments").cast(IntegerType()))
df_video = df_video.withColumn("Views", col("Views").cast(IntegerType()))

print("Campos Likes, Comments e Views convertidos para Integer em 'df_video'.")
df_video.printSchema() # Verificando a mudança

Campos Likes, Comments e Views convertidos para Integer em 'df_video'.
root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)



# N Converter e Renomear em 'df_comentario'

In [51]:
# Converter os tipos para Inteiro e Renomear 'Likes' para 'Likes Comment'

# Converte 'Likes' e 'Sentiment' para IntegerType
df_comentario = df_comentario.withColumn("Likes", col("Likes").cast(IntegerType()))
df_comentario = df_comentario.withColumn("Sentiment", col("Sentiment").cast(IntegerType()))

# Renomeia a coluna 'Likes' para 'Likes Comment'
df_comentario = df_comentario.withColumnRenamed("Likes", "Likes Comment")

print("Campos 'Likes' e 'Sentiment' convertidos para Integer, e 'Likes' renomeado para 'Likes Comment' em 'df_comentario'.")
df_comentario.printSchema()

Campos 'Likes' e 'Sentiment' convertidos para Integer, e 'Likes' renomeado para 'Likes Comment' em 'df_comentario'.
root
 |-- _c0: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Likes Comment: integer (nullable = true)
 |-- Sentiment: integer (nullable = true)



# Criar campo 'Interaction'

In [52]:
# Adiciona uma nova coluna chamada Interacion, que é a soma das interações (Likes, Comments, Views) do vídeo
df_video = df_video.withColumn("Interacion", col("Likes") + col("Comments") + col("Views"))

print("coluna 'interacion' criada. Visualizando as 5 primeiras linhas:")
df_video.select("Likes","Comments","Views","Interacion").show(5)

coluna 'interacion' criada. Visualizando as 5 primeiras linhas:
+------+--------+--------+----------+
| Likes|Comments|   Views|Interacion|
+------+--------+--------+----------+
|910553|   81975|52061447|  53053975|
| 32092|     181|  425850|    458123|
|    16|       3|   83121|     83140|
|  8534|     517|  254621|    263672|
| 90491|    1633| 2854238|   2946362|
+------+--------+--------+----------+
only showing top 5 rows



# Converter 'Published At' para 'date'

In [53]:
# Converte a coluna Published At para o tipo Date (apenas data)
df_video = df_video.withColumn("Published At", to_date(col("Published At")))

print("Coluna 'published at' convertida para Date")
df_video.printSchema()

Coluna 'published at' convertida para Date
root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interacion: integer (nullable = true)



# Criar campo 'Year'

In [54]:
# Adiciona uma coluna Year extraindo o ano da coluna Published At
df_video = df_video.withColumn("Year", year(col("Published At")))

print("coluna 'year' criada. Visualizando as primeiras 5 linhas:")
df_video.select("Published At", "Year").show(5)

coluna 'year' criada. Visualizando as primeiras 5 linhas:
+------------+----+
|Published At|Year|
+------------+----+
|  2020-08-18|2020|
|  2021-12-03|2021|
|  2022-08-18|2022|
|  2022-08-18|2022|
|  2021-04-22|2021|
+------------+----+
only showing top 5 rows



# Mesclar (Join) df_comentario e df_video

In [55]:
# Combina o DataFrame principal (df_video) com o DataFrame de comentários (df_comentario) usando a chave Video ID. O tipo left garante que todos os vídeos sejam mantidos, mesmo que não tenham comentários correspondentes
df_join_video_comments = df_video.join(df_comentario, on="Video ID", how="left")

print("junção (left join) entre 'df_video' e 'df_comentario' concluída.")
df_join_video_comments.printSchema()

junção (left join) entre 'df_video' e 'df_comentario' concluída.
root
 |-- Video ID: string (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interacion: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- _c0: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Likes Comment: integer (nullable = true)
 |-- Sentiment: integer (nullable = true)



# Ler 'USvideos.csv'

In [56]:
# Lendo o terceiro arquivo
df_us_videos = spark.read.csv("/content/USvideos.csv", header=True, inferSchema=True)

print("Leitura do 'USvideos.csv' concluída. Schema:")
df_us_videos.printSchema()

Leitura do 'USvideos.csv' concluída. Schema:
root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



# Mesclar (Join) df_us_videos e df_video

In [58]:
# Renomeia colunas em df_us_videos para evitar ambiguidade e garantir a junção correta
df_us_videos_processed = df_us_videos \
    .withColumnRenamed("title", "Title") \
    .withColumnRenamed("views", "us_views") \
    .withColumnRenamed("likes", "us_likes") \
    .withColumnRenamed("comment_count", "us_comment_count") \
    .withColumnRenamed("video_id", "us_video_id") \
    .withColumnRenamed("dislikes", "us_dislikes")

# Realiza a junção usando o df_us_videos_processed. Agora 'Title' corresponde e outras colunas são únicas.
df_join_video_usvideos = df_video.join(df_us_videos_processed, on="Title", how="left")

print("junção (left join) entre 'df_videos' e 'df_us_videos' concluida.")
print("visualizando o resultado do join (colunas selecionadas):")

# Seleciona colunas, referindo-se explicitamente a 'Views' de df_video, e o novo 'us_views' se necessário.
df_join_video_usvideos.select("Title", col("Views"), col("trending_date")).show(5, truncate=False)

junção (left join) entre 'df_videos' e 'df_us_videos' concluida.
visualizando o resultado do join (colunas selecionadas):
+-----------------------------------------------------------------------------------------------------+--------+-------------+
|Title                                                                                                |Views   |trending_date|
+-----------------------------------------------------------------------------------------------------+--------+-------------+
|ASMR MUKBANG DOUBLE BIG MAC &amp; CHEESY HASH BROWNS &amp; CHICKEN NUGGETS (No Talking) EATING SOUNDS|17975269|NULL         |
|Deadly car bomb detonates outside Moscow                                                             |808787  |NULL         |
|How Biden&#39;s student loan forgiveness program will work                                           |97434   |NULL         |
|Celebrating My 400 Pound Milestone.... McDonald&#39;s Mukbang                                        |5283664 |NULL

In [ ]:
# Renomeia colunas em df_us_videos para evitar ambiguidade e garantir a junção correta
df_us_videos_processed = df_us_videos \
    .withColumnRenamed("title", "Title") \
    .withColumnRenamed("views", "us_views") \
    .withColumnRenamed("likes", "us_likes") \
    .withColumnRenamed("comment_count", "us_comment_count") \
    .withColumnRenamed("video_id", "us_video_id") \
    .withColumnRenamed("dislikes", "us_dislikes")

# Realiza a junção usando o df_us_videos_processed. Agora 'Title' corresponde e outras colunas são únicas.
df_join_video_usvideos = df_video.join(df_us_videos_processed, on="Title", how="left")

print("junção (left join) entre 'df_videos' e 'df_us_videos' concluida.")
print("visualizando o resultado do join (colunas selecionadas):")

# Seleciona colunas, referindo-se explicitamente a 'Views' de df_video, e o novo 'us_views' se necessário.
df_join_video_usvideos.select("Title", col("Views"), col("trending_date")).show(5, truncate=False)

junção (left join) entre 'df_videos' e 'df_us_videos' concluida.
visualizando o resultado do join (colunas selecionadas):
+-----------------------------------------------------------------------------------------------------+--------+-------------+
|Title                                                                                                |Views   |trending_date|
+-----------------------------------------------------------------------------------------------------+--------+-------------+
|ASMR MUKBANG DOUBLE BIG MAC &amp; CHEESY HASH BROWNS &amp; CHICKEN NUGGETS (No Talking) EATING SOUNDS|17975269|NULL         |
|Deadly car bomb detonates outside Moscow                                                             |808787  |NULL         |
|How Biden&#39;s student loan forgiveness program will work                                           |97434   |NULL         |
|Celebrating My 400 Pound Milestone.... McDonald&#39;s Mukbang                                        |5283664 |NULL

# Verificar nulos em 'df_video'

In [ ]:
print("contagem dos valores nulos por coluna em 'df_video':")
df_video.select([count(when(col(c).isNull(), c)).alias(c) for c in df_video.columns]).show() # Executa uma contagem de nulos para cada coluna no df_video após todos os tratamentos, exibindo um resumo da qualidade dos dados

contagem dos valores nulos por coluna em 'df_video':
+---+-----+--------+------------+-------+-----+--------+-----+
|_c0|Title|Video ID|Published At|Keyword|Likes|Comments|Views|
+---+-----+--------+------------+-------+-----+--------+-----+
|  0|    0|       0|           0|      0|    2|       2|    2|
+---+-----+--------+------------+-------+-----+--------+-----+



# Salvar 'df_video' tratado

In [ ]:
if "_c0" in df_video.columns:
  df_video = df_video.drop("-c0") # Condicionalmente remove a coluna _c0 (coluna de índice frequentemente criada durante a exportação/leitura de CSVs) do df_video se ela existir
  print("coluna '_c0' removida de 'df_video'.")
else:
    print("coluna '_c0' não encontrada em 'df_video', nada a remover.")

# Salva o DataFrame df_video limpo e transformado no formato Parquet (otimizado e colunar) no diretório videos-tratados-parquet, sobrescrevendo qualquer arquivo anterior
df_video.write.mode("overwrite").parquet("videos-tratados-parquet")
print("DataFrame 'df_video' salvo com sucesso como 'videos-tratados-parquet'.")


coluna '_c0' removida de 'df_video'.
DataFrame 'df_video' salvo com sucesso como 'videos-tratados-parquet'.


# Salvar 'df_join_video_comments' tratado

In [ ]:
if "_c0" in df_join_video_comments.columns:
  df_join_video_comments = df_join_video_comments.drop("_c0") # Faz a mesma remoção condicional da coluna _c0 no DataFrame resultante da junção
  print("Coluna '_c0' removida de 'df_join_video_comments'.")
else:
  print("Coluna '_c0' não encontrada em df_join_video_comments, nada a remover.")

# Salvando o DataFrame de join
df_join_video_comments.write.mode("overwrite").parquet("videos-comments-tratados-parquet") # Salva o DataFrame resultado da junção (df_join_video_comments) no formato Parquet

print("DataFrame 'df_join_video_comments' salvo com sucesso como 'videos-comments-tratados-parquet'.")

Coluna '_c0' removida de 'df_join_video_comments'.
DataFrame 'df_join_video_comments' salvo com sucesso como 'videos-comments-tratados-parquet'.
